In [1]:
# coding: utf-8
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'sans-serif'
import pandas as pd
import numpy as np
import multiprocessing
import joblib
from pandas.core.indexes.datetimes import date_range
from time import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgboost

In [2]:
data2 = pd.read_csv("./한국남부발전(주)_신인천소내 태양광발전실적_20200820.csv",encoding='CP949',index_col=0,parse_dates=True)
data2 = data2.loc[:'2019']
preprocessed_data2 = data2[['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']]
preprocessed_data2.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']
preprocessed_redata2 = preprocessed_data2[['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']]
preprocessed_redata2 = preprocessed_redata2.reindex(pd.date_range(start='2013-01-01', end='2019-12-31'), fill_value=0)
preprocessed_redata2.loc['2016-05-04'] = preprocessed_redata2.loc['2016-05-03'].values
preprocessed_redata2.loc['2016-06-01'] = preprocessed_redata2.loc['2016-06-02'].values
# preprocessed_redata2['position'] = '신인천소내'

data3 = pd.read_csv("./한국남부발전(주)_부산복합자재창고 태양광발전실적_20200924.csv",encoding='CP949',index_col=0,parse_dates=True)
data3_1 = data3[data3['호기'] == 1]
data3_1 = data3_1.loc[:'2019']
preprocessed_data3_1 = data3_1[['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']]
preprocessed_data3_1.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']
preprocessed_redata3_1 = preprocessed_data3_1[['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']]
# preprocessed_redata3_1['position'] = '부산복합자재창고'

data4 = pd.read_csv("./한국남부발전(주)_부산신항 태양광발전실적_20200820.csv",encoding='CP949',index_col=0,parse_dates=True)
data4 = data4.loc[:'2019']
preprocessed_data4 = data4[['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']]
preprocessed_data4.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']
preprocessed_redata4 = preprocessed_data4[['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']]
# preprocessed_redata4['position'] = '부산신항'

preprocessed_redata2_flat = preprocessed_redata2.stack(dropna=False)
preprocessed_redata3_1_flat = preprocessed_redata3_1.stack(dropna=False)
preprocessed_redata4_flat = preprocessed_redata4.stack(dropna=False)

index_date =pd.date_range(start='2013-01-01 00:00:00', end='2020-01-01 00:00:00', closed='right', freq='H', tz='Asia/Seoul')
print(len(preprocessed_redata2_flat.index))
print(len(preprocessed_redata3_1_flat.index))
print(len(preprocessed_redata4_flat.index))
print(len(index_date))

df_pre2 = pd.DataFrame(data=preprocessed_redata2_flat.values, index=index_date)
df_pre2['site'] = 2
df_pre3_1 = pd.DataFrame(data=preprocessed_redata3_1_flat.values, index=index_date)
df_pre3_1['site'] = 3
df_pre4 = pd.DataFrame(data=preprocessed_redata4_flat.values, index=index_date)
df_pre4['site'] = 4

df_full = pd.concat([df_pre2, df_pre3_1, df_pre4], axis=0)
df_full.columns = ['발전량', 'site']

61344
61344
61344
61344


In [3]:
generations_data = df_full
# generations_data = pd.DataFrame(index=df_full.index, data=df_full.values)
# generations_data.columns = ['발전량']
generations_data = generations_data.fillna(0)
selected_gens = generations_data
selected_gens

,발전량,site
2013-01-01 01:00:00+09:00,0.0,2
2013-01-01 02:00:00+09:00,0.0,2
2013-01-01 03:00:00+09:00,0.0,2
2013-01-01 04:00:00+09:00,0.0,2
2013-01-01 05:00:00+09:00,0.0,2
...,...,...
2019-12-31 20:00:00+09:00,0.0,4
2019-12-31 21:00:00+09:00,0.0,4
2019-12-31 22:00:00+09:00,0.0,4
2019-12-31 23:00:00+09:00,0.0,4


In [4]:
weather_data_2 = pd.read_csv('37.4772_126.6249_Solcast_PT60M.csv', parse_dates=True, index_col='PeriodEnd')
weather_data_3 = pd.read_csv('35.10468_129.0323_Solcast_PT60M.csv', parse_dates=True, index_col='PeriodEnd')
weather_data_4 = pd.read_csv('35.10468_129.0323_Solcast_PT60M.csv', parse_dates=True, index_col='PeriodEnd')
weather_data_2['site'] = 2
weather_data_3['site'] = 3
weather_data_4['site'] = 4
weather_data_full = pd.concat([weather_data_2, weather_data_3, weather_data_4], axis=0)
weather_data_full

,PeriodStart,Period,AirTemp,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,Ebh,Ghi,...,GtiTracking,PrecipitableWater,RelativeHumidity,SnowWater,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,AlbedoDaily,site
PeriodEnd,,,,,,,,,,,,,,,,,,,,,
2012-12-31 02:00:00+00:00,2012-12-31T01:00:00Z,PT60M,-8.6,-149,9.1,-14.9,155,513,197,352,...,477,2.8,60.0,1.0,1010.5,317,4.4,68,0.20,2
2012-12-31 03:00:00+00:00,2012-12-31T02:00:00Z,PT60M,-7.3,-163,6.1,-14.3,156,647,296,453,...,513,3.0,57.4,1.0,1010.3,313,4.7,63,0.20,2
2012-12-31 04:00:00+00:00,2012-12-31T03:00:00Z,PT60M,-6.4,-178,5.3,-13.7,165,665,322,487,...,499,3.3,56.3,1.0,1010.0,308,4.9,61,0.20,2
2012-12-31 05:00:00+00:00,2012-12-31T04:00:00Z,PT60M,-5.7,166,6.7,-13.1,166,617,290,456,...,492,3.6,56.3,1.0,1009.5,303,5.0,62,0.20,2
2012-12-31 06:00:00+00:00,2012-12-31T05:00:00Z,PT60M,-5.2,152,24.3,-12.4,234,166,71,306,...,326,3.8,56.7,1.0,1009.1,298,5.1,67,0.20,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-01 18:00:00+00:00,2021-01-01T17:00:00Z,PT60M,1.9,-74,7.6,-4.8,0,0,0,0,...,0,5.3,60.9,0.0,1016.6,294,2.9,151,0.15,4
2021-01-01 19:00:00+00:00,2021-01-01T18:00:00Z,PT60M,1.8,-86,44.1,-4.7,0,0,0,0,...,0,5.5,62.1,0.0,1016.7,304,2.5,138,0.15,4
2021-01-01 20:00:00+00:00,2021-01-01T19:00:00Z,PT60M,1.7,-94,30.5,-4.5,0,0,0,0,...,0,5.6,63.2,0.0,1017.0,312,2.7,126,0.15,4


In [5]:
X = weather_data_full.drop(columns=['DewpointTemp','WindSpeed10m','WindDirection10m', 'RelativeHumidity','PrecipitableWater','SnowWater','SurfacePressure', 'GtiFixedTilt','GtiTracking', 'AlbedoDaily'])
y = selected_gens
X, y

(                                    PeriodStart Period  AirTemp  Azimuth  \
 PeriodEnd                                                                  
 2012-12-31 02:00:00+00:00  2012-12-31T01:00:00Z  PT60M     -8.6     -149   
 2012-12-31 03:00:00+00:00  2012-12-31T02:00:00Z  PT60M     -7.3     -163   
 2012-12-31 04:00:00+00:00  2012-12-31T03:00:00Z  PT60M     -6.4     -178   
 2012-12-31 05:00:00+00:00  2012-12-31T04:00:00Z  PT60M     -5.7      166   
 2012-12-31 06:00:00+00:00  2012-12-31T05:00:00Z  PT60M     -5.2      152   
 ...                                         ...    ...      ...      ...   
 2021-01-01 18:00:00+00:00  2021-01-01T17:00:00Z  PT60M      1.9      -74   
 2021-01-01 19:00:00+00:00  2021-01-01T18:00:00Z  PT60M      1.8      -86   
 2021-01-01 20:00:00+00:00  2021-01-01T19:00:00Z  PT60M      1.7      -94   
 2021-01-01 21:00:00+00:00  2021-01-01T20:00:00Z  PT60M      1.5     -102   
 2021-01-01 22:00:00+00:00  2021-01-01T21:00:00Z  PT60M      1.3     -110   

In [6]:
X_new = X
X_new.index = X.index.tz_convert(tz='Asia/Seoul') # forecast
X_new = X_new.drop(['PeriodStart', 'Period'], axis=1) # forecast
X_new = X_new.loc['2013-01-01 01:00:00+09:00':'2020-01-01 00:00:00+09:00']
y_new = y
X_new.shape, y_new.shape

((184032, 9), (184032, 2))

In [7]:
X_new.index

DatetimeIndex(['2013-01-01 01:00:00+09:00', '2013-01-01 02:00:00+09:00',
               '2013-01-01 03:00:00+09:00', '2013-01-01 04:00:00+09:00',
               '2013-01-01 05:00:00+09:00', '2013-01-01 06:00:00+09:00',
               '2013-01-01 07:00:00+09:00', '2013-01-01 08:00:00+09:00',
               '2013-01-01 09:00:00+09:00', '2013-01-01 10:00:00+09:00',
               ...
               '2019-12-31 15:00:00+09:00', '2019-12-31 16:00:00+09:00',
               '2019-12-31 17:00:00+09:00', '2019-12-31 18:00:00+09:00',
               '2019-12-31 19:00:00+09:00', '2019-12-31 20:00:00+09:00',
               '2019-12-31 21:00:00+09:00', '2019-12-31 22:00:00+09:00',
               '2019-12-31 23:00:00+09:00', '2020-01-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', name='PeriodEnd', length=184032, freq=None)

In [8]:
y_new.index

DatetimeIndex(['2013-01-01 01:00:00+09:00', '2013-01-01 02:00:00+09:00',
               '2013-01-01 03:00:00+09:00', '2013-01-01 04:00:00+09:00',
               '2013-01-01 05:00:00+09:00', '2013-01-01 06:00:00+09:00',
               '2013-01-01 07:00:00+09:00', '2013-01-01 08:00:00+09:00',
               '2013-01-01 09:00:00+09:00', '2013-01-01 10:00:00+09:00',
               ...
               '2019-12-31 15:00:00+09:00', '2019-12-31 16:00:00+09:00',
               '2019-12-31 17:00:00+09:00', '2019-12-31 18:00:00+09:00',
               '2019-12-31 19:00:00+09:00', '2019-12-31 20:00:00+09:00',
               '2019-12-31 21:00:00+09:00', '2019-12-31 22:00:00+09:00',
               '2019-12-31 23:00:00+09:00', '2020-01-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', length=184032, freq=None)

In [9]:
X_new['month'] = X_new.index.month
X_new['day'] = X_new.index.day
X_new['hour'] = X_new.index.hour
X_new['dayofyear'] = X_new.index.dayofyear

In [10]:
X_new.loc['2016-03':'2016-12']['dayofyear'] =  X_new.loc['2016-03':'2016-12']['dayofyear'] - 1
dropThis2016 = pd.date_range(start='2016-02-29 00:00:00+09:00', end='2016-02-29 23:00+09:00', freq='H')
dropThis2016

DatetimeIndex(['2016-02-29 00:00:00+09:00', '2016-02-29 01:00:00+09:00',
               '2016-02-29 02:00:00+09:00', '2016-02-29 03:00:00+09:00',
               '2016-02-29 04:00:00+09:00', '2016-02-29 05:00:00+09:00',
               '2016-02-29 06:00:00+09:00', '2016-02-29 07:00:00+09:00',
               '2016-02-29 08:00:00+09:00', '2016-02-29 09:00:00+09:00',
               '2016-02-29 10:00:00+09:00', '2016-02-29 11:00:00+09:00',
               '2016-02-29 12:00:00+09:00', '2016-02-29 13:00:00+09:00',
               '2016-02-29 14:00:00+09:00', '2016-02-29 15:00:00+09:00',
               '2016-02-29 16:00:00+09:00', '2016-02-29 17:00:00+09:00',
               '2016-02-29 18:00:00+09:00', '2016-02-29 19:00:00+09:00',
               '2016-02-29 20:00:00+09:00', '2016-02-29 21:00:00+09:00',
               '2016-02-29 22:00:00+09:00', '2016-02-29 23:00:00+09:00'],
              dtype='datetime64[ns, pytz.FixedOffset(540)]', freq='H')

In [11]:
X_renew = X_new.drop(dropThis2016)
y_renew = y_new.drop(dropThis2016)
X_selected = X_renew
y_selected = y_renew
X_selected, y_selected

(                           AirTemp  Azimuth  CloudOpacity  Dhi  Dni  Ebh  Ghi  \
 PeriodEnd                                                                       
 2013-01-01 01:00:00+09:00     -8.6        4           0.0    0    0    0    0   
 2013-01-01 02:00:00+09:00     -9.4      -40           0.0    0    0    0    0   
 2013-01-01 03:00:00+09:00    -10.1      -66           0.9    0    0    0    0   
 2013-01-01 04:00:00+09:00     -9.8      -81          12.0    0    0    0    0   
 2013-01-01 05:00:00+09:00     -8.7      -91          32.3    0    0    0    0   
 ...                            ...      ...           ...  ...  ...  ...  ...   
 2019-12-31 20:00:00+09:00      0.7      101           0.0    0    0    0    0   
 2019-12-31 21:00:00+09:00      0.5       94           0.0    0    0    0    0   
 2019-12-31 22:00:00+09:00      0.3       85           0.0    0    0    0    0   
 2019-12-31 23:00:00+09:00      0.1       72           0.0    0    0    0    0   
 2020-01-01 00:0

In [12]:
X_selected, y_selected

(                           AirTemp  Azimuth  CloudOpacity  Dhi  Dni  Ebh  Ghi  \
 PeriodEnd                                                                       
 2013-01-01 01:00:00+09:00     -8.6        4           0.0    0    0    0    0   
 2013-01-01 02:00:00+09:00     -9.4      -40           0.0    0    0    0    0   
 2013-01-01 03:00:00+09:00    -10.1      -66           0.9    0    0    0    0   
 2013-01-01 04:00:00+09:00     -9.8      -81          12.0    0    0    0    0   
 2013-01-01 05:00:00+09:00     -8.7      -91          32.3    0    0    0    0   
 ...                            ...      ...           ...  ...  ...  ...  ...   
 2019-12-31 20:00:00+09:00      0.7      101           0.0    0    0    0    0   
 2019-12-31 21:00:00+09:00      0.5       94           0.0    0    0    0    0   
 2019-12-31 22:00:00+09:00      0.3       85           0.0    0    0    0    0   
 2019-12-31 23:00:00+09:00      0.1       72           0.0    0    0    0    0   
 2020-01-01 00:0

In [13]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_selected[X_selected['site'] == 2], y_selected[y_selected['site'] == 2], test_size=0.14285714285714285714, shuffle=False)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_selected[X_selected['site'] == 3], y_selected[y_selected['site'] == 3], test_size=0.14285714285714285714, shuffle=False)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_selected[X_selected['site'] == 4], y_selected[y_selected['site'] == 4], test_size=0.14285714285714285714, shuffle=False)
X_train = np.concatenate((X_train1, X_train2, X_train3), axis=0)
y_train = np.concatenate((y_train1, y_train2, y_train3), axis=0)
X_test = np.concatenate((X_test1, X_test2, X_test3), axis=0)
y_test = np.concatenate((y_test1, y_test2, y_test3), axis=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((157680, 13), (26280, 13), (157680, 2), (26280, 2))

In [14]:
X_train = np.ascontiguousarray(X_train)
y_train = np.ascontiguousarray(y_train)

In [15]:
multi_xgb = MultiOutputRegressor(xgboost.XGBRegressor(n_jobs = multiprocessing.cpu_count(),n_estimators = 400, max_depth = 4, learning_rate = 0.1))
fit_result = multi_xgb.fit(X_train, y_train)

/Users/itd/opt/anaconda3/envs/fbprophet/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


In [16]:
y_pred_multixgb = multi_xgb.fit(X_train, y_train).predict(X_test).reshape(-1,2)
df_y_pred_multixgb = pd.DataFrame(data=y_pred_multixgb, columns=['발전량','site'])
df_y_test = pd.DataFrame(data=y_test, columns=['발전량', 'site'])

/Users/itd/opt/anaconda3/envs/fbprophet/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
/Users/itd/opt/anaconda3/envs/fbprophet/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


In [17]:
error_xgb = pd.DataFrame(data=np.zeros(y_pred_multixgb.shape), columns=['발전량','site'])
error_xgb['발전량'] = df_y_test['발전량'] - df_y_pred_multixgb['발전량']
error_xgb['site'] = df_y_test['site']
abserror_xgb = abs(error_xgb)

In [20]:
np.mean(abserror_xgb[abserror_xgb['site'] == 2.0])

발전량     5.815661
site    2.000000
dtype: float64

In [21]:
np.mean(abserror_xgb[abserror_xgb['site'] == 3.0])

발전량     5.406705
site    3.000000
dtype: float64

In [22]:
np.mean(abserror_xgb[abserror_xgb['site'] == 4.0])

발전량     2.530994
site    4.000000
dtype: float64

In [ ]:
joblib.dump(fit_result, "xgboost.pickle")

In [ ]:
load = joblib.load('xgboost.pickle')
load.estimator

In [ ]:
df_y_pred_multixgb

In [ ]:
df_y_pred_multixgb['발전량'].loc['2019-01-01'].plot()